In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

def IntegerSafe(value): # In case there are non-integer type to be converted.
    try:
        return int(value)
    except:
        return None

ss = SparkSession.builder.getOrCreate()
sc = ss.sparkContext


business = sc.textFile("../Data/SF_business/filtered_registered_business_sf.csv")\
             .map(lambda x : x.split(','))\
             .map(lambda x : (IntegerSafe(x[0]), x[1], x[2], x[3], x[4]))
              

schema = StructType([ StructField("zip", IntegerType(), True),
                      StructField("name", StringType(), False),
                      StructField("street", StringType(), True),
                      StructField("city", StringType(), True),
                      StructField("state", StringType(), True)
                    ])

business_df = ss.createDataFrame(business, schema)

22/02/22 19:03:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/22 19:03:54 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [18]:
from pyspark.sql.functions import *

## Create a UDF called check_sf which checks whether a given value contains “San Francisco” or “SF”.


In [19]:
check_sf= udf(lambda x: ('San Francisco' in x) or ('SF' in x),BooleanType())

In [21]:
business_df.select('name',
                   check_sf(business_df['street']).alias('check')).show(5)

+--------------------+-----+
|                name|check|
+--------------------+-----+
|   Tournahu George L|false|
|Stephens Institut...|false|
|Stephens Institut...|false|
|Stephens Institut...|false|
|Stephens Institut...|false|
+--------------------+-----+
only showing top 5 rows



Traceback (most recent call last):
  File "/Users/fanli/opt/anaconda3/envs/adv630/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/Users/fanli/opt/anaconda3/envs/adv630/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/fanli/opt/anaconda3/envs/adv630/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 663, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/fanli/opt/anaconda3/envs/adv630/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [22]:
def check(x):
    if ('San Francisco' in x) or ('SF' in x):
        return True
    else:
        return False

In [25]:
check_sf = udf(check)
business_df.select('name',
                  check_sf(business_df['city']).alias('check')).show(5)

+--------------------+-----+
|                name|check|
+--------------------+-----+
|   Tournahu George L| true|
|Stephens Institut...| true|
|Stephens Institut...| true|
|Stephens Institut...| true|
|Stephens Institut...| true|
+--------------------+-----+
only showing top 5 rows



Traceback (most recent call last):
  File "/Users/fanli/opt/anaconda3/envs/adv630/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/Users/fanli/opt/anaconda3/envs/adv630/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/fanli/opt/anaconda3/envs/adv630/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 663, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/fanli/opt/anaconda3/envs/adv630/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [15]:
check_sf = udf(lambda x: ('San Francisco' in x) or ('SF' in x),BooleanType())

## Apply check_sf to business_df to check whether city is San Francisco.


In [26]:
business_df.select('name',check_sf('city').alias('a')).show(5)

+--------------------+----+
|                name|   a|
+--------------------+----+
|   Tournahu George L|true|
|Stephens Institut...|true|
|Stephens Institut...|true|
|Stephens Institut...|true|
|Stephens Institut...|true|
+--------------------+----+
only showing top 5 rows



In [28]:
ss.stop()